In [1]:
import pandas as pd
import numpy as np
from prog.tools import *
from prog.model import *
import warnings, pickle, os, json
warnings.filterwarnings("ignore")

aluminum_division = 12
random_state = 99
weight_limit = 56
l_extra = 6.4
f_extra = 8

In [2]:
input_ = {
    "work_id": "julietest",
    "op": 1,
    "model_id": "20230807120000000",
    "speed": "590",
    "l_angle_ori": "35",
    "l_weight_ori": "79",
    "f_angle_ori": "75",
    "f_weight_ori": "155",
    "material": "aluminum", # 'aluminum'
}

In [3]:
work_id      = input_["work_id"]
op           = input_["op"]
model_id     = input_["model_id"]
speed        = int(input_["speed"])
l_angle_ori  = int(input_["l_angle_ori"])
l_weight_ori = float(input_["l_weight_ori"])
f_angle_ori  = int(input_["f_angle_ori"])
f_weight_ori = float(input_["f_weight_ori"])
material     = input_["material"]

In [4]:
root = r"C:\Users\tzuli\Documents\python\teco"

# 取得predict位置
pred_path = os.path.join(root, "data", "predict")        
os.makedirs(pred_path, exist_ok = True)
output_json = os.path.join(pred_path, "output.json")

# 取得model位置     
model_detail = os.path.join(root, "data", "train", model_id, "model")

config_path = os.path.join(root, "prog", "config.json")
with open(config_path) as f:
    config = json.load(f)

### Load model

In [5]:
side = "L"
model_l = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

side = "F"
model_f = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

In [6]:
df_l = pd.DataFrame([[l_angle_ori, l_weight_ori]], columns = ["初始_L側角度", "初始_L側不平衡量"])
df_f = pd.DataFrame([[f_angle_ori, f_weight_ori]], columns = ["初始_F側角度", "初始_F側不平衡量"])

In [7]:
df_l, df_f

(   初始_L側角度  初始_L側不平衡量
 0       35       79.0,
    初始_F側角度  初始_F側不平衡量
 0       75      155.0)

In [8]:
# 超過60補上特定值
if df_l.loc[0, "初始_L側不平衡量"] > weight_limit:
    df_l.loc[0, "初始_L側不平衡量"] += l_extra

if df_f.loc[0, "初始_F側不平衡量"] > weight_limit:
    df_f.loc[0, "初始_F側不平衡量"] += f_extra

In [9]:
df_l, df_f

(   初始_L側角度  初始_L側不平衡量
 0       35       85.9,
    初始_F側角度  初始_F側不平衡量
 0       75      163.0)

In [10]:
angle_init = np.linspace(0, 360, (aluminum_division + 1)).astype(int)
print(f"angle_init = {angle_init}")

df_l = calculate_angle_proportion(df_l, angle_init, aluminum_division, side = "L")            
df_f = calculate_angle_proportion(df_f, angle_init, aluminum_division, side = "F")

angle_init = [  0  30  60  90 120 150 180 210 240 270 300 330 360]


In [11]:
df_l = calculate_weight(df_l, side = "L")
df_f = calculate_weight(df_f, side = "F")

In [12]:
df_l.head()

,初始_L側角度,初始_L側不平衡量,angle_proportion,angle_0,angle_target,3_raw,7_raw,5_raw,10_raw,3_full,...,7,5,10,3_solution,7_solution,5_solution,10_solution,37_delta,5_delta,10_delta
0,35,85.9,0.166667,0.0,37,26.0,60.0,43.0,86.0,0.0,...,4,43,30,"{'6': '1', '20': '1'}",{'4': '1'},"{'7': '1', '16': '1', '20': '1'}","{'7': '2', '16': '1'}",0.0,0.0,0.0


In [13]:
df_f.head()

,初始_F側角度,初始_F側不平衡量,angle_proportion,angle_0,angle_target,3_raw,7_raw,5_raw,10_raw,3_full,...,7,5,10,3_solution,7_solution,5_solution,10_solution,37_delta,5_delta,10_delta
0,75,163.0,0.5,0.0,55,49.0,114.0,82.0,163.0,0.0,...,0,26,51,"{'4': '1', '5': '1', '20': '2'}",{},"{'6': '1', '20': '1'}","{'4': '1', '7': '1', '20': '2'}",-2.0,0.0,0.0


In [14]:
df_l1 = encoding(df_l, side = "L")
df_f1 = encoding(df_f, side = "F")

In [15]:
X_l, y_l = split_data(df_l1, random_state, predict = True)
X_f, y_f = split_data(df_f1, random_state, predict = True)

In [16]:
X_l.head()

,angle_proportion,angle_0,3,7,37_delta,5,5_delta,10,10_delta
0,0.166667,0.0,0.302678,0.046566,0.0,0.500582,0.0,0.349243,0.0


In [17]:
X_f.head()

,angle_proportion,angle_0,3,7,37_delta,5,5_delta,10,10_delta
0,0.5,0.0,0.300613,0.0,-2.0,0.159509,0.0,0.312883,0.0


In [18]:
y_pred_l = model_l.predict(X_l)[0]
y_pred_f = model_f.predict(X_f)[0]
print(f"real - l: {y_l.values[0]}, f: {y_f.values[0]}")
print(f"pred - l: {y_pred_l}, f: {y_pred_f}")

real - l: 0, f: 1
pred - l: 0, f: 1


In [19]:
l_ans = df_l.iloc[0]
f_ans = df_f.iloc[0]

proportion = {    
    "1:0": ["10"],
    "5:5": ["5", "5"],
    "3:7": ["3", "7"]
}
predicts = {}
for key, value in proportion.items():
    l_weight_pred = [str(round((l_ans[f"{value[0]}_full"] * weight_limit) + l_ans[value[0]]))]
    f_weight_pred = [str(round((f_ans[f"{value[0]}_full"] * weight_limit) + f_ans[value[0]]))]

    if value != ["10"]:
        l_weight_pred += [str(round((l_ans[f"{value[1]}_full"] * weight_limit) + l_ans[value[1]]))]
        f_weight_pred += [str(round((f_ans[f"{value[1]}_full"] * weight_limit) + f_ans[value[1]]))]
    predict = {
        "l_angle_pred":  str(l_angle_ori),
        "l_weight_pred": l_weight_pred,
        "l_comb": {},
        "f_angle_pred":  str(f_angle_ori),
        "f_weight_pred": f_weight_pred,
        "f_comb": {},
    }

    fill_solution(value, l_ans, predict, l_weight_pred, weight_limit, side = "l")
    fill_solution(value, f_ans, predict, f_weight_pred, weight_limit, side = "f")

    predicts[key] = predict
    
predicts

{'1:0': {'l_angle_pred': '35',
  'l_weight_pred': ['86'],
  'l_comb': {'86': {'56': {'count': '1', 'comb': {'20': '2', '16': '1'}},
    '30': {'count': '1', 'comb': {'7': '2', '16': '1'}}}},
  'f_angle_pred': '75',
  'f_weight_pred': ['163'],
  'f_comb': {'163': {'56': {'count': '2', 'comb': {'20': '2', '16': '1'}},
    '51': {'count': '1', 'comb': {'4': '1', '7': '1', '20': '2'}}}}},
 '5:5': {'l_angle_pred': '35',
  'l_weight_pred': ['43', '43'],
  'l_comb': {'43': {'43': {'count': '1',
     'comb': {'7': '1', '16': '1', '20': '1'}}}},
  'f_angle_pred': '75',
  'f_weight_pred': ['82', '82'],
  'f_comb': {'82': {'56': {'count': '1', 'comb': {'20': '2', '16': '1'}},
    '26': {'count': '1', 'comb': {'6': '1', '20': '1'}}}}},
 '3:7': {'l_angle_pred': '35',
  'l_weight_pred': ['26', '60'],
  'l_comb': {'26': {'26': {'count': '1', 'comb': {'6': '1', '20': '1'}}},
   '60': {'56': {'count': '1', 'comb': {'20': '2', '16': '1'}},
    '4': {'count': '1', 'comb': {'4': '1'}}}},
  'f_angle_pred':

In [20]:
y_map = {0: "3:7", 1: "5:5", 2: "1:0"}
best = {
    "l_side": y_map[y_pred_l],
    "f_side": y_map[y_pred_f]
}
best

{'l_side': '3:7', 'f_side': '5:5'}

In [21]:
result = {
    "status":   "success",
    "reason":   "",
    "work_id":  work_id,
    "op":       op,
    "model_id": model_id,
    "predict":  predicts,
    "best": best
}
result

{'status': 'success',
 'reason': '',
 'work_id': 'julietest',
 'op': 1,
 'model_id': '20230807120000000',
 'predict': {'1:0': {'l_angle_pred': '35',
   'l_weight_pred': ['86'],
   'l_comb': {'86': {'56': {'count': '1', 'comb': {'20': '2', '16': '1'}},
     '30': {'count': '1', 'comb': {'7': '2', '16': '1'}}}},
   'f_angle_pred': '75',
   'f_weight_pred': ['163'],
   'f_comb': {'163': {'56': {'count': '2', 'comb': {'20': '2', '16': '1'}},
     '51': {'count': '1', 'comb': {'4': '1', '7': '1', '20': '2'}}}}},
  '5:5': {'l_angle_pred': '35',
   'l_weight_pred': ['43', '43'],
   'l_comb': {'43': {'43': {'count': '1',
      'comb': {'7': '1', '16': '1', '20': '1'}}}},
   'f_angle_pred': '75',
   'f_weight_pred': ['82', '82'],
   'f_comb': {'82': {'56': {'count': '1', 'comb': {'20': '2', '16': '1'}},
     '26': {'count': '1', 'comb': {'6': '1', '20': '1'}}}}},
  '3:7': {'l_angle_pred': '35',
   'l_weight_pred': ['26', '60'],
   'l_comb': {'26': {'26': {'count': '1', 'comb': {'6': '1', '20': 

In [22]:
with open(output_json, 'w') as file:
    json.dump(result, file, indent = 4)